In [1]:
print('Hello Sunny!!!')

Hello Sunny!!!


In [2]:
import os
os.chdir('../')

In [3]:
%pwd

'd:\\placement\\ML_Projects\\Gen_ai\\Medical Bot\\DocLine'

In [4]:
# from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [6]:
extracted_data = load_pdf("data/")

In [7]:
# extracted_data

In [8]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [9]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 5860


In [10]:
from langchain.embeddings import HuggingFaceEmbeddings

In [11]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [12]:
embeddings = download_hugging_face_embeddings()

C:\Users\Asus\AppData\Local\Temp\ipykernel_32168\1337643473.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\Asus\.conda\envs\doc\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [14]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [15]:
# query_result

In [16]:
from dotenv import load_dotenv

load_dotenv()

True

In [17]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')

In [18]:
import os
print(os.getenv("PINECONE_API_KEY"))  # Should print the key

pcsk_mWRkf_FaPYGJmmzdj9omoLNghTuBDgw58k7hLsgKoncP5vcXSe31jzyzkkhuA3p3Joh3A


In [21]:
# import pinecone 
# from pinecone.grpc import PineconeGRPC as Pinecone
# from pinecone import ServerlessSpec
# import os

# pc = Pinecone(api_key="PINECONE_API_KEY")

# index_name = "test"

# pc.create_index(
#     name=index_name,
#     dimension=384, # Replace with your model dimensions
#     metric="cosine", # Replace with your model metric
#     spec=ServerlessSpec(
#         cloud="aws",
#         region="us-east-1"
#     ) 
# )

In [22]:
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

{
    "name": "medicalbot",
    "metric": "cosine",
    "host": "medicalbot-mj7ucm2.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [27]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [28]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [29]:
docsearch

In [30]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [31]:
retrieved_docs = retriever.invoke("What is Acne?")

In [32]:
retrieved_docs

[Document(id='5bc3cd15-6493-476e-93d4-4d982e6d6ba6', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='36b28000-3a4f-4480-bef9-7f929fac96e4', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM -

In [33]:
from dotenv import load_dotenv

load_dotenv()

True

In [34]:
KEY=os.getenv("GROQ_API_KEY")

In [36]:
from langchain_groq import ChatGroq
llm = ChatGroq(groq_api_key=KEY,model="Gemma2-9b-It",temperature=0.5)

In [37]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000017CB4014220>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000017CB40155D0>, model_name='Gemma2-9b-It', temperature=0.5, model_kwargs={}, groq_api_key=SecretStr('**********'))

In [85]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


# system_prompt = (
#     "You are an assistant for question-answering tasks. "
#     "Use the following pieces of retrieved context to answer "
#     "the question. If you don't know the answer, say that you "
#     "don't know. Use three sentences maximum and keep the "
#     "answer concise."
#     "\n\n"
#     "{context}"
# )
system_prompt = (
    "👋 Hey there! I'm your AI medical assistant, here to help with **diseases, symptoms, treatments, and medical conditions**. 🏥💙 "
    "If your question is **not** related to health, I might not have the right answer. "
    "In that case, I'll gently remind you: 🤗 *'I can only assist with medical-related queries. Please ask about diseases, symptoms, or treatments.'* "
    "Otherwise, I'll use the best medical knowledge to provide clear and helpful information! ✅✨"
    "\n\n"
    "{context}"
)



prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [86]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [87]:
response = rag_chain.invoke({"input": "what is Acne?"})
print(response["answer"])

Acne is a common skin condition that causes pimples on the face, chest, and back.  

Here's a breakdown:

* **What causes it?**  Your pores get clogged with oil (sebum), dead skin cells, and bacteria.
* **Who gets it?**  It's very common, affecting millions of people in the US.  It can happen at any age, but it's most common during puberty.


Let me know if you'd like to know more about acne, such as types of acne, treatments, or how to prevent it. 



In [88]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])

It looks like you've provided some information about a complete blood count (CBC) and Sudden Unexpected Death (SUD) in athletes. 

**Stats** is short for **statistics**. 

It's a branch of mathematics that deals with collecting, analyzing, interpreting, presenting, and organizing data. 

The text you provided uses statistics to:

* **Describe the frequency of SUD:**  "The yearly rate for occurrence of SUD in people less than 35 years of age is less than seven incidents per 100,000."
* **Compare the proportion of exercise-related SUD:** "Of all SUD cases, only about 8% are exercise related."


Let me know if you have any other questions about blood counts, SUD, or statistics! 




In [89]:
response = rag_chain.invoke({"input": "What i do for acne?"})
print(response["answer"])

It looks like you're asking about ways to manage acne. Based on the information you provided, here are some tips:

* **Hygiene:** Wash your face often and keep your hair off your face. 
* **Diet:** Eat a balanced diet and try to identify any foods that seem to trigger your acne flare-ups.
* **Sun Exposure:**  Moderate sun exposure might help, but always consult a doctor before using sunlight as a treatment.
* **Avoid Picking:**  Don't pick or squeeze blemishes, as this can worsen inflammation and lead to scarring.
* **Stress Management:** Find healthy ways to manage stress, as it can contribute to acne.

**Supplements and Herbal Remedies:**

The text mentions some supplements and herbs that are sometimes used for acne:

* **Milk Thistle:**  May help protect the liver.
* **Essential Fatty Acids:**  Like omega-3s, could help reduce inflammation.
* **Vitamin B Complex, Zinc, Vitamin A, and Chromium:** These nutrients might play a role in skin health.
* **Cnidium Seed and Honeysuckle Flowe

In [90]:
response = rag_chain.invoke({"input": "tell mw a joke?"})
print(response["answer"])

I can only assist with medical-related queries. Please ask about diseases, symptoms, or treatments. 


